In [1]:
%load_ext watermark
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.colors
from matplotlib.colors import LinearSegmentedColormap, ListedColormap
import seaborn as sns

import session_config
import reports
import geospatial
import gridforecast as gfcast
import logging
import userdisplay as disp
from myst_nb import glue
from IPython.display import display, Markdown



logging.basicConfig(
    filename='app.log', 
    level=logging.DEBUG,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)

logger = logging.getLogger(__name__)


# available data
surveys = session_config.collect_survey_data()
codedefs = pd.read_csv('data/end_process/codes.csv').set_index('code')

o_dates = {'start':'2020-01-01', 'end':'2021-12-31'}
prior_dates = {'start':'2015-11-15', 'end':'2019-12-31'}

# all data
canton = 'Bern'
d= surveys.reset_index(drop=True)
lake_params = {'canton':canton, 'date_range':o_dates, 'feature_type': 'l'}
lake_params_p = {'canton':canton, 'date_range':prior_dates, 'feature_type':'l'}

# all surveys lakes, rivers combined
c_all_l,  _ = gfcast.filter_data(d,{'canton': canton, 'feature_type': 'l',  'date_range': {'start':prior_dates['start'], 'end':o_dates['end']}})
call_l_surveys, call_l_land = gfcast.make_report_objects(c_all_l)

# prior data does not include locations in canton
o_prior_l = d[(d.canton != canton)&(d['date'] <= prior_dates['end'])&(d.feature_type == 'l')].copy()
o_report_l, o_land_use_l = gfcast.make_report_objects(o_prior_l)
lake_results = gfcast.reports_and_forecast(lake_params,lake_params_p , ldata=d.copy(), logger=logger, other_data=o_land_use_l.df_cat)

# Use cases

How each stake holder interprets the data is dependent on the use case. Here we present the answers to some of the most common questions that can be answered using the report or the methods desccribed in this document.

For an example we will use the results from the lakes in Bern.

## Is their more now than there was before ?

__How do you know ?__ The best way to verify is to take more samples. However, the previous results in the canton and the results from locations outside the canton with similar land-use cahracteristics are good indicators.

From the report we can identify several key pieces of information that will help us make the determination.

(use-case-observed)=
__Consider the observed results__

ref: [Bern vital statistics](bern-vital-statistics), ref: [Bern most common](bern-most-common)

1. Vital statisitics: The average pcs/m of trash was greater in 2021 than 2017-2019
2. The combined pcs/m of the most common objects (74% of all objects) is 2.3 pcs/m. That is .4 pcs/m greater than all the objects from 2017-2019

(use-case-forecast)=
__Consider the forecasted values__

ref: [Bern forecasts](bern-forecasts), ref: [Empirical Bayes](gridforecaster), ref: [Forecast class](forecast-class)

1. Given the 99th percenitle of 2021, we expect the values for 2025 to be approximately equal 2017-2019
2. If we assume the maximum value of 2021 was not a mistake, on average we expect to find more in 2025
3. Considering random samples from locations outside of the canton from locations with similar land-cover we expect to find more

__Conclusion:__ The expected average survey result for lakes in the canton of Bern is expected to increase from 1.96 pcs/m in 2017 - 2019 to 3.06 in 2025. The campaign of 2017-2019 only had 24 samples versus 74 in 2021. Given that the sampling campaigns were managed by different organizations and were initiated for different purposes we can not be sure about the magnitude of the increase. However, the results from other lakes in Switzerland suggest that __an increase is more likely than a decrease__.

__Notes:__ Comparing the sample totals is not the best way to make this type of analysis. We know that 74% of the total can be accounted for by 15 different objects. Comparing like items or grouping like items would provide a more accurate representation of the expected sample results for 2025.

__Recommended actions:__ Set a sampling strategy for the canton. Collect at least as many samples from the same lakes as 2021 and be sure that a portion of those samples come from the same locations as 2021.

---

(cleanest-beach)=
## Where is the cleanest beach ?

__How do you find the cleanest locations ?__ We can make a determination based on the sampling history or we can use the land-use profile to define the characteristics of what the cleanest beach may look like.

__Use the sampling history__

ref: [Bern municipal results](bern-municipal-results)

The municipalites with lowest average pcs/m and more than three samples is __Lüscherz with 0.75 pcs/m (Bielersee) and Spiez with 0.79 pcs/m (Thunersee)__.

__Use the land use profile__

ref: [Bern rate per feature](bern-rate-per-feature), ref: [land-use](calculating-land-use), ref: [Land use classe](landusereporter)

Survey locations with at least 40% - 60% of the land attributed to forest or undefined use have the lowest average pcs/m. Together these two land-use categories account for 50% of samples. Notice that urban locations (80% - 100% buildings) also have a very low average result. This is the inverse of what is expected in general.

__Conclusion:__ The two locations with the lowest observed average pcs/m fit the expected land-use profile. However, the results in urban locations may be of interest given that that the average results is 50% less than we would find in other cantons under the same land-profile.

__Notes:__ The sampling profile should approximate the profile of the canton. For example if the region is mainly agricultural and all the samples come from the urban centers then the sampling is not representative of the envrironment. 

__Recommended actions:__ Set a sampling strategy for the canton. With local and regional stakeholders identify the sampling locations that best represent the geography of the canton.

---

## How much of the items are single use plastics (SUP) ?

__How are you defining single use plastics ?__ The definition of SUP has not been established by federal or cantonal authorities. Here we consider the obvious choices and exclude cigarette end and other tobaco products.

__Single use plastics:__

1. PET bottles
2. Plastic bottle caps
3. Straws, stirrers
4. Plastic cotton swabs
5. Fast food or to go containters
6. Plastic bags
7. Snack wrappers
8. Fireworks
9. Lollypop sticks
10. Cups, lids, single use foamed and hard plastics
11. Medical; containers/tubes/ packaging
12. Diapers, wipes and sanitary pads

__Use the object inventory__

ref: [Survey reports](surveyreporter)

The object inventory is included with each report. It gives the summary result of all objects identified from the surveys (below). By adding up the % of total of the all the objects identified as single use plastics we find that on the shores of lakes in Bern __14% (1190/8423) of the trash was single use plastics__.

__Conclusion:__ SUP are about 14% of the total found in 2021. Which means that approximately 80% of the objects found are not classified as SUP. 

__Notes:__ This finding is in agreement with massive amount of pressure on consumers to do the the right thing. Furthermore, municipalities invest a siginifigant portion of their budgets in maintaining the shoreline of lakes.

__Recommended actions:__ Stakeholders should define a list of objects that can be standardized and takes into account observations in the field.

__Object inventory Bern lakes, single use plastics__

In [2]:
pd.set_option('display.max_rows', 200)

bern_report = lake_results['this_report']
objects = bern_report.object_summary()
objects['description'] = objects.index.map(lambda x: codedefs['en'].loc[x])
objects = objects[['description', *objects.columns[:-1]]]
objects = objects[objects.quantity > 0]
ooi = ['G10', 'G100', 'Gcaps', 'G95', 'G30', 'G31', 'G33', 'G34', 'G35', 'G8', 'G7', 'G6', 'G5', 'G4', 'G37', 'G101', 'G2', 'G96', 'G98']
obj_oi = objects.loc[ooi].copy()
obj_oi.set_index('description', drop=True, inplace=True)
obj_oi = obj_oi[['quantity', 'pcs/m', '% of total', 'rate']]
obj_oi.rename(columns={'rate':'fail rate'}, inplace=True)
glue('object-inv', obj_oi.round(2), display=False)

```{glue} object-inv
```
---

## How much of the items can you identify ?

Preventing or mittigating plastics in the environment depends on how well you can identify the objects that are present.

__Unidentified items: Gfrags, Gfoams__

ref: [Bern most common](bern-most-common)

There are two categories that are specifically for objects that are found but can not be identified. Note that if an object is found AND it can be indentified but it does not _fit_ into and existing category a new category is created.

__Conclusion:__ __About 20% of the objects found in the canton were unidentified.__ 

__Notes:__ The correct idenitification of objects is a team effort. Surveyors with little experience tend to have a greater percentage of unidentified objects in their surveys.

__Recommended actions:__ Stakeholders should define a list of objects that can be standardized and take into account observations in the field. Encourage joint operations with surveyors from different regions. Items that are rare in one location maybe fairly common elsewhere.

---


(canton-common)=
## What objects are common to all cantons ?

ref: [Combined cantons](combined-most-common), ref: [Bern most common](bern-most-common)

__Use the most common items__

For each report their is a detail of the most common items. If we use the combined results this will tell you, given the results from all the lakes and the cantons under consideration, exactly what the most common items are.
::::{grid} 1 2 2 2 

:::{grid-item-card}

__Common to all cantons__

1. Fragmented plastics 
2. Cigarette filters 
3. Expanded polystyrene 
4. Food wrappers; candy, snacks 	
5. Industrial sheeting 	
6. Industrial pellets (nurdles) 	
7. plastic caps, lid rings: G21, G22, G23, G24 	
8. Glass drink bottles, pieces
9. Foam packaging/insulation/polyurethane 
10. Cotton bud/swab sticks 	
11. Plastic construction waste 
12. Metal bottle caps, lids & pull tabs from cans 	
13. Tobacco; plastic packaging, containers 	
14. Foil wrappers, aluminum foil 	
15. Straws and stirrers

:::

:::{grid-item-card}

__Common to Bern__

1. Cigarette filters
2. Fragmented plastics 
3. Industrial sheeting 
4. Food wrappers; candy, snacks 
5. Expanded polystyrene 
6. Glass drink bottles, pieces
7. Packaging films nonfood or unknown
8. Foam packaging/insulation/polyurethane
9. plastic caps, lid rings: G21, G22, G23, G24
10. Industrial pellets (nurdles)
11. Plastic construction waste 
12. Fireworks; rocket caps, exploded parts & packaging 	
13. Cotton bud/swab sticks 	
14. Tobacco; plastic packaging, containers 	
15. Foil wrappers, aluminum foil

::: 
::::


__Conclusion:__ __14/15 of the most common items in Bern are also common to the rest of the cantons.__

__Notes:__ This may be more pertinent for cantons that have a common river or lake.

__Recommended actions:__ Neighbor cantons should consult the most common items in their respective areas to find possible points of collaboration.

---

## What is the chance of finding X ?

How often an object or category of objects is found is an excellent indeicator of the distribution of the item in the environment.

__Use the most common or the object inventory__

ref: [Combined cantons](combined-most-common), ref: [Forecast class](forecast-class)

__How likely are you to find:__

1. PET bottle on the shore of a lake in Bern: __about 7% of surveys will yield a plastic bottle__
2. Plastic bottle top: __about 65% of surveys will yield a plastic bottle top__
3. Diaper of babywipe: __22% of surveys will yield a diaper or babywipe__


__Conclusion:__ The standard report will tell you how likely you are to find at least one an object.

__Notes:__ This is a simple calculation using the most likely estimate of a binomial distribution.

__Recommended actions:__ None